**XML to CSV**

# Import Libraries

In [1]:
import pandas as pd
import xml.etree.cElementTree as xet
from glob import glob
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2,InceptionV3,InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

# Data Preprocessing

In [2]:
#data path
path=glob('data/annotations/*.xml')

In [3]:
# preprocessing the data for training
abels_dict=dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:
    #filename=path[0]
    info=xet.parse(filename)
    root=info.getroot()
    member_object=root.find('object')
    labels_info=member_object.find('bndbox')
    xmin=int(labels_info.find('xmin').text)
    xmax=int(labels_info.find('xmax').text)
    ymin=int(labels_info.find('ymin').text)
    ymax=int(labels_info.find('ymax').text)
    #print(xmin,xmax,ymin,ymax)
    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [4]:
# converting data in csv file
df=pd.DataFrame(labels_dict)
df.head(5)

,filepath,xmin,xmax,ymin,ymax
0,data/annotations/Cars213.xml,150,233,160,194
1,data/annotations/Cars207.xml,254,372,229,264
2,data/annotations/Cars398.xml,72,196,139,222
3,data/annotations/Cars367.xml,224,382,161,222
4,data/annotations/Cars401.xml,158,238,223,240


In [5]:
# saving data in csv file
df.to_csv('labels.csv',index=False)

In [6]:
# function to return filename with path
def getFilename(filename):
    filename_image=xet.parse(filename).getroot().find('filename').text
    filepath_image=os.path.join('data/images',filename_image)
    return filepath_image

In [7]:
# storing the image path
image_path=list(df['filepath'].apply(getFilename))

In [8]:
labels=df.iloc[:,1:].values

In [9]:
data=[]
output=[]
for ind in range(len(image_path)):
    image=image_path[ind]
    img_arr=cv2.imread(image)
    h,w,d=img_arr.shape
    #preprocessing
    load_image=load_img(image,target_size=(224,224))
    load_image_arr=img_to_array(load_image)
    norm_load_image_array=load_image_arr/255
    #normalization
    xmin,xmax,ymin,ymax=labels[ind]
    nxmin,nxmax=xmin/w,xmax/w
    nymin,nymax=ymin/h,ymax/h
    label_norm=(nxmin,nxmax,nymin,nymax)
    #append
    data.append(norm_load_image_array)
    output.append(label_norm)

In [10]:
X=np.array(data,dtype=np.float32) # converting to numpy array
y=np.array(output,dtype=np.float32)
X.shape,y.shape

((433, 224, 224, 3), (433, 4))

In [11]:
# train test split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

**Deep Learning Model**

In [19]:
# model
iception_resnet=InceptionResNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))
iception_resnet.trainable=False
#---------------
headmodel=iception_resnet.output
headmodel=Flatten()(headmodel)
headmodel=Dense(500,activation='relu')(headmodel)
headmodel=Dropout(0.2)(headmodel)
headmodel=Dense(250,activation='relu')(headmodel)
headmodel=Dropout(0.1)(headmodel)
headmodel=Dense(4,activation='sigmoid')(headmodel)
#------
model=Model(inputs=iception_resnet.input,outputs=headmodel)

In [20]:
#compilt model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_813 (Conv2D)            (None, 111, 111, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_812 (Batch  (None, 111, 111, 32  96         ['conv2d_813[0][0]']             
 Normalization)                 )                                                           

 Normalization)                                                                                   
                                                                                                  
 activation_816 (Activation)    (None, 52, 52, 192)  0           ['batch_normalization_816[0][0]']
                                                                                                  
 max_pooling2d_18 (MaxPooling2D  (None, 25, 25, 192)  0          ['activation_816[0][0]']         
 )                                                                                                
                                                                                                  
 conv2d_821 (Conv2D)            (None, 25, 25, 64)   12288       ['max_pooling2d_18[0][0]']       
                                                                                                  
 batch_normalization_820 (Batch  (None, 25, 25, 64)  192         ['conv2d_821[0][0]']             
 Normaliza

In [21]:
#training
tfb=TensorBoard('object_detection')

In [22]:
# training the model
history=model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                 validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
35/35 [==============================] - 58s 2s/step - loss: 0.1032 - val_loss: 0.0524
Epoch 2/100
35/35 [==============================] - 45s 1s/step - loss: 0.0617 - val_loss: 0.0246
Epoch 3/100
35/35 [==============================] - 49s 1s/step - loss: 0.0290 - val_loss: 0.0183
Epoch 4/100
35/35 [==============================] - 44s 1s/step - loss: 0.0177 - val_loss: 0.0168
Epoch 5/100
35/35 [==============================] - 42s 1s/step - loss: 0.0142 - val_loss: 0.0141
Epoch 6/100
35/35 [==============================] - 42s 1s/step - loss: 0.0125 - val_loss: 0.0127
Epoch 7/100
35/35 [==============================] - 43s 1s/step - loss: 0.0102 - val_loss: 0.0139
Epoch 8/100
35/35 [==============================] - 43s 1s/step - loss: 0.0095 - val_loss: 0.0130
Epoch 9/100
35/35 [==============================] - 42s 1s/step - loss: 0.0088 - val_loss: 0.0118
Epoch 10/100
35/35 [==============================] - 42s 1s/step - loss: 0.0077 - val_loss: 0.0117
Epoch 11/

In [24]:
# retraining the model
history=model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=100)

Epoch 101/200
35/35 [==============================] - 28s 812ms/step - loss: 0.0046 - val_loss: 0.0143
Epoch 102/200
35/35 [==============================] - 27s 785ms/step - loss: 0.0043 - val_loss: 0.0130
Epoch 103/200
35/35 [==============================] - 27s 788ms/step - loss: 0.0043 - val_loss: 0.0124
Epoch 104/200
35/35 [==============================] - 28s 790ms/step - loss: 0.0042 - val_loss: 0.0122
Epoch 105/200
35/35 [==============================] - 27s 779ms/step - loss: 0.0047 - val_loss: 0.0124
Epoch 106/200
35/35 [==============================] - 27s 781ms/step - loss: 0.0043 - val_loss: 0.0118
Epoch 107/200
35/35 [==============================] - 27s 775ms/step - loss: 0.0047 - val_loss: 0.0113
Epoch 108/200
35/35 [==============================] - 27s 773ms/step - loss: 0.0050 - val_loss: 0.0111
Epoch 109/200
35/35 [==============================] - 27s 774ms/step - loss: 0.0045 - val_loss: 0.0111
Epoch 110/200
35/35 [==============================] - 27s 779ms

In [25]:
# saving the model
model.save('my_mode.h5')